In [ ]:
#Iniciando com a instalação da ADK
%pip -q install google-adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/

In [ ]:
#Importação dos módulos genai e types
from google import genai
from google.genai import types

#Importação das classes Agent, Runner, InMemorySessionService e google_search
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search

#Importação do módulo de datas, para trabalhar com datas
from datetime import date

#Importação do textwrap, para manipular quebra de linhas e formatar textos
import textwrap

#Importação das funções display e Markdown, para estilizações
from IPython.display import display, Markdown

#Importação da biblioteca request, para interação HTTP
import requests

#Importação do módulo warning, para controle de avisos e função pra ignorar avisos.
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Configura a API Key do Google Gemini

import os

from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [ ]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [ ]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [ ]:
##########################################
# --- Agente 1: Buscador de Dicas --- #
##########################################

def agente_buscador(topico, data_de_hoje):
    buscador = Agent(
        name="agente_buscador",
        model="gemini-2.0-flash",
        description="Agente que busca informações sobre treino para alunos de nível avançado.",
        tools=[google_search],
        instruction="""
        Você é um assistente de pesquisas, que está auxiliando um aluno que possui uma vasta experiência com exercícios físicos e profundo conhecedor da prática de atividade física. Sua tarefa é usar a ferramenta de busca do google (google_search)para pesquisar por informações sobre treinos, atividades físicas, benefícios para a saúde através da atividade física e assuntos correlacionados ao mundo fitness, inclusive, dicas para o dia a dia saudável. Não seja prolixo, traga informações relevantes, de fácil compreensão e sempre traga informações com embassamento científico (não complique a cabeça do nosso aluno, embora de nível avançado, ele é experiente e pode, inclusive, querer te ensinar rs). Se um conteúdo buscado não tiver muita qualidade, ou seja, pouco acesso, baixa relevância e sem embasamento científico, ignore e busque algo melhor. Essas informações devem ser mostradas em português brasileiro (se tiver algo de outro país, faça a tradução para o português do Brasil para que possa ser utilizado), de no máximo 3 meses antes da data de hoje. Se limite a trazer até 10 tópicos no conteúdo buscado.
        """
    )
    entrada_do_agente_buscador = f"Tópico: {topico}\nData de hoje: {data_de_hoje}"

    # Executa o agente
    lancamentos_buscados = call_agent(buscador, entrada_do_agente_buscador)
    return lancamentos_buscados

In [ ]:
################################################
# --- Agente 2: Planejador de treinos --- #
################################################
def agente_planejador(topico, lancamentos_buscados):
    planejador = Agent(
        name="agente_planejador",
        model="gemini-2.0-flash",
        # Inserir as instruções do Agente Planejador #################################################
        instruction="""
        Você é um planejador de conteúdo, especialista em atividade física, mundo fitness, educação física e personal trainner. Com base na lista de lançamentos mais recentes e relevantes buscados, você deve:	Usar a ferramenta de busca do Google (google_search) para criar um plano de treinamentos e prática de atividade física, sobre quais são os pontos mais relevantes que poderiamos abordar compor um plano de treinamento de atividades físicas. Você também pode usar o (google_search) para encontrar mais informações sobre os temas e aprofundar. Ao final, você irá escolher de três até sete práticas de atividades físicas, seus seus pontos mais relevantes, e um plano com esses tópicos a serem abordados em um planto de treinos que será desenvolvido posteriormente.
        """,
        description="Agente que planeja treinamentos",
        tools=[google_search]
    )

    entrada_do_agente_planejador = f"Tópico:{topico}\nLançamentos buscados: {lancamentos_buscados}"
    # Executa o agente
    plano_do_post = call_agent(planejador, entrada_do_agente_planejador)
    return plano_do_post

In [ ]:
######################################
# --- Agente 3: Prescritor do Treinos --- #
######################################
def agente_redator(topico, plano_de_post):
    redator = Agent(
        name="agente_redator",
        model="gemini-2.0-flash",
        instruction="""
            Você é um prescritor de treinos, especialista em mundo fitness e educação física. Você prescreve treinos para a empresa FitNerd, a maior rede de academias do Brasil. Utilize o material fornecido no plano de treino e os pontos mais relevantes fornecidos e, com base nisso, escreva um plano de treino que inclua a execução perfeita da atividade física e as melhores práticas a serem adotadas para este plano em um rascunho que será utilizado na academia FitNerd. O plano deve ser engajador, informativo, com linguagem simples e incluir de 7 até 10 exercícios.
            """,
        description="Agente redator de planos de treino para a academia FitNerd"
    )
    entrada_do_agente_redator = f"Tópico: {topico}\nPlano de post: {plano_de_post}"
    # Executa o agente
    rascunho = call_agent(redator, entrada_do_agente_redator)
    return rascunho

In [ ]:
##########################################
# --- Agente 4: Revisor de Qualidade --- #
##########################################
def agente_revisor(topico, rascunho_gerado):
    revisor = Agent(
        name="agente_revisor",
        model="gemini-2.0-flash",
        instruction="""
            Você é um Editor e Revisor de Conteúdo meticuloso, especializado em academias, prática de atividades física, mundo fitness e educação física. Você está atendendo alunos do nível avançado, logo possuem uma excelente compreensão de termos de academia, estão investindo muito em tempo e resultados, pois pretendem competir em grandes dispustas do estilo Mr Olimpia. Revise o rascunho do plano de treinamento da academia "FitNerd" verificando clareza, concisão, correção e tom. Se o plano de treino estiver bom, responda apenas 'O plano está ótimo e pronto para ser gerado!'. Caso haja problemas, aponte-os e sugira melhorias.
            """,
        description="Agente revisor de plano de treinamentos para a FitNerd."
    )
    entrada_do_agente_revisor = f"Tópico: {topico}\nRascunho: {rascunho_gerado}"
    # Executa o agente
    texto_revisado = call_agent(revisor, entrada_do_agente_revisor)
    return texto_revisado

In [ ]:
data_de_hoje = date.today().strftime("%d/%m/%Y")

print("🚀 Iniciando o Sistema de apoio ao aluno iniciante da FitNerd com 4 Agentes 🚀")

# --- Obter o Tópico do Usuário ---
topico = input("❓ Por favor, digite o TÓPICO sobre o qual você quer gerar o seu treino: ")

# Inserir lógica do sistema de agentes ################################################
if not topico:
	print("Você esqueceu de digitar o tópico!")
else:
	print (f"Maravilha! Vamos então vamos criar o seu plano: {topico}")

lancamentos_buscados = agente_buscador(topico, data_de_hoje)
print("\n--- Resultados do Agente 1 (Buscador) ---\n")
display(to_markdown(lancamentos_buscados))
print("----------------------------------------------------")

plano_de_post = agente_planejador(topico, lancamentos_buscados)
print("\n--- Resultado do Agente 2 (Planejador) ---\n")
display(to_markdown(plano_de_post))
print("----------------------------------------------------")

rascunho_de_post = agente_redator(topico, plano_de_post)
print("\n--- 📝 Resultado do Agente 3 (Redator) ---\n")
display(to_markdown(rascunho_de_post))
print("--------------------------------------------------------------")

post_final = agente_revisor(topico, rascunho_de_post)
print("\n--- 📝 Resultado do Agente 4 (Revisor) ---\n")
display(to_markdown(post_final))
print("--------------------------------------------------------------")

🚀 Iniciando o Sistema de apoio ao aluno iniciante da FitNerd com 4 Agentes 🚀
❓ Por favor, digite o TÓPICO sobre o qual você quer gerar o seu treino: Treinos para pernas
Maravilha! Vamos então vamos criar o seu plano de treinamento Treinos para pernas

--- Resultados do Agente 1 (Buscador) ---



> Olá! Tudo bem? Para te ajudar com treinos de pernas, preparei algumas opções focadas em resultados e otimização, considerando seu nível avançado. Veja o que encontrei:
> 
>  
> Considerando seu nível avançado, aqui estão algumas sugestões para turbinar seus treinos de perna, com foco em hipertrofia e força:
> 
>  1. **Priorize a intensidade:** A intensidade é crucial para resultados em hipertrofia, mesmo em fases de maior volume.
>  2. **Atenção à execução:** A execução correta é fundamental, especialmente em exercícios como agachamento, levantamento terra e avanço. Ajustes na técnica podem fazer grande diferença.
>  3. **Carga adequada:** Não exagere nas cargas. Priorize menos carga e mais repetições em estímulos metabólicos, sempre com foco na técnica.
>  4. **Periodicidade:** Ajuste o número de treinos de perna por semana conforme a periodização, geralmente entre uma ou duas vezes.
>  5. **Mobilidade:** Inclua exercícios de mobilidade antes do treino para diminuir dores, melhorar a performance e evitar lesões. Exemplos: agachamento lateral alternado, extensão de quadril e mobilidade de quadril, joelho e tornozelo.
> 
>  **Sugestões de exercícios avançados:**
> 
>  *   **Agachamento Livre:** Exercício fundamental para trabalhar diversos músculos das pernas e glúteos.
>  *   **Leg Press:** Variação que permite trabalhar com cargas elevadas, focando em quadríceps, glúteos e isquiotibiais.
>  *   **Stiff:** Ótimo para trabalhar os isquiotibiais e glúteos.
>  *   **Avanço com Halteres (Passadas):** Exercício unilateral que trabalha equilíbrio, coordenação e força nas pernas.
>  *   **Cadeira Extensora:** Para isolar o quadríceps.
>  *   **Mesa Flexora:** Para isolar os isquiotibiais.
>  *   **Elevação Pélvica (com barra):** Para trabalhar glúteos e isquiotibiais.
>  *   **Panturrilha em Pé:** Para fortalecer a panturrilha.
>  *   **Levantamento Terra:** Ativa os músculos da posterior da coxa, glúteos, região lombar e abdominal, promovendo resistência e ganho de massa muscular.
>  *   **Agachamento Búlgaro:** Trabalha o quadríceps e glúteos, promovendo fortalecimento e hipertrofia.
> 
>  **Dicas extras:**
> 
>  *   **Drop-Set:** Utilize essa técnica para aumentar a intensidade do treino. Exemplo: Cadeira Extensora unilateral: 4 séries de 5 repetições máximas, diminuindo o peso a cada série, sem intervalo.
>  *   **Isometria:** Adicione isometria em exercícios como a cadeira adutora, segurando por 3 segundos em cada repetição.
>  *   **Controle a fase excêntrica:** Foque no controle da descida (fase excêntrica) dos exercícios, alongando o tempo sob tensão.
> 
>  **Frequência e Volume:**
> 
>  *   **Avançados:** 3-4 vezes por semana, com pelo menos 1 dia de descanso entre os treinos.
>  *   **Volume:** 5 séries de 5-8 repetições por exercício.
>  *   **Carga:** Pesada, quase levando os músculos à falha.
> 
>  Lembre-se de que a individualidade é crucial. Adapte as sugestões às suas necessidades e objetivos, e sempre consulte um profissional de educação física para garantir a segurança e eficácia do seu treino.
> 


----------------------------------------------------

--- Resultado do Agente 2 (Planejador) ---



> Com base nas informações fornecidas, podemos criar um plano de treinamento de pernas avançado, enfatizando hipertrofia e força. Para compor este plano, abordaremos os seguintes pontos relevantes e práticas de atividades físicas:
> 
> **Pontos Relevantes a Abordar no Plano de Treino:**
> 
> 1.  **Intensidade e Carga Adequada:** Priorizar a intensidade é crucial, mas sem exagerar nas cargas. O foco deve estar na execução correta e no estímulo metabólico, ajustando a carga para permitir mais repetições com boa técnica.
> 
> 2.  **Execução e Técnica:** Atenção rigorosa à execução correta dos exercícios, especialmente nos movimentos complexos como agachamento, levantamento terra e avanço. Ajustes na técnica podem otimizar os resultados e prevenir lesões.
> 
> 3.  **Periodicidade e Frequência:** Ajustar a frequência dos treinos de perna conforme a periodização, geralmente entre 3-4 vezes por semana para níveis avançados, garantindo pelo menos um dia de descanso entre os treinos.
> 
> 4.  **Mobilidade e Aquecimento:** Incluir exercícios de mobilidade antes do treino para melhorar a amplitude de movimento, diminuir dores e evitar lesões. Exemplos: agachamento lateral alternado, extensão de quadril e mobilidade de quadril, joelho e tornozelo.
> 
> 5.  **Técnicas Avançadas:** Incorporar técnicas como drop-set, isometria e controle da fase excêntrica para aumentar a intensidade e o tempo sob tensão, maximizando o estímulo muscular.
> 
> **Práticas de Atividades Físicas para Incluir no Plano de Treino:**
> 
> 1.  **Agachamento Livre:** Exercício fundamental para trabalhar diversos músculos das pernas e glúteos, promovendo força e hipertrofia.
> 
> 2.  **Leg Press:** Variação que permite trabalhar com cargas elevadas, focando em quadríceps, glúteos e isquiotibiais.
> 
> 3.  **Stiff:** Ótimo para trabalhar os isquiotibiais e glúteos, contribuindo para o fortalecimento da cadeia posterior.
> 
> 4.  **Avanço com Halteres (Passadas):** Exercício unilateral que trabalha equilíbrio, coordenação e força nas pernas, além de ativar os músculos estabilizadores.
> 
> 5.  **Levantamento Terra:** Ativa os músculos da posterior da coxa, glúteos, região lombar e abdominal, promovendo resistência e ganho de massa muscular.
> 
> **Plano de Treino Proposto:**
> 
> Este plano será estruturado com base nos pontos relevantes e práticas de atividades físicas acima. Cada treino deve começar com exercícios de mobilidade e aquecimento, seguidos pelos exercícios principais com foco na técnica e intensidade. As técnicas avançadas, como drop-set e isometria, serão incorporadas estrategicamente para maximizar os resultados. A frequência será de 3-4 vezes por semana, com volume de 5 séries de 5-8 repetições por exercício, utilizando cargas pesadas que levem os músculos à falha.
> 
> Este plano de treino será desenvolvido de forma mais detalhada posteriormente, levando em consideração a individualidade e os objetivos específicos do praticante.
> 


----------------------------------------------------

--- 📝 Resultado do Agente 3 (Redator) ---



> ## Rascunho do Plano de Treino FitNerd - Pernas de Aço! 💪
> 
> **Fala, galera da FitNerd!** Preparados para um treino de pernas que vai te deixar andando bambo? 🤪 Preparamos um rascunho de um plano *power* para quem busca hipertrofia, força e, claro, aquela definição que a gente ama! 😉
> 
> **Lembrem-se:** Esse é um *rascunho*. Antes de começar, converse com um profissional da FitNerd para adaptar o treino às suas necessidades e objetivos, ok? 😎
> 
> **🚀 Aquecimento (5-10 minutos):**
> 
> *   **Agachamento Lateral Alternado:** 2 séries de 10 repetições (cada lado).
> *   **Extensão de Quadril:** 2 séries de 15 repetições (cada perna).
> *   **Mobilidade de Quadril, Joelho e Tornozelo:** Movimentos circulares e alongamentos dinâmicos.
> 
> **🏋️‍♀️ Treino (3-4 vezes por semana):**
> 
> **Foco na Execução:** Prestem atenção na postura, amplitude do movimento e contração muscular. A técnica perfeita é a chave para evitar lesões e maximizar os resultados! 😉
> 
> **Carga:** Escolham um peso que desafie vocês, mas que permita completar as repetições com boa forma. O objetivo é sentir o músculo trabalhando! 🔥
> 
> **Descanso:** Entre as séries, descansem de 60 a 90 segundos para recuperar a energia e manter a intensidade do treino.
> 
> **Exercícios:**
> 
> 1.  **Agachamento Livre:** 5 séries de 5-8 repetições. 
>     *   **Dica FitNerd:** Mantenha a coluna reta, o abdômen contraído e desça até a linha dos joelhos. Use um espelho para conferir a postura!
> 2.  **Leg Press:** 5 séries de 5-8 repetições.
>     *   **Dica FitNerd:** Ajuste o banco para que seus joelhos formem um ângulo de 90 graus na descida. Empurre com a força dos calcanhares!
> 3.  **Stiff (com barra ou halteres):** 5 séries de 5-8 repetições.
>     *   **Dica FitNerd:** Mantenha as pernas semi-flexionadas e incline o tronco para frente, sentindo alongar a parte de trás da coxa. Concentre-se na conexão mente-músculo!
> 4.  **Avanço com Halteres (Passadas):** 3 séries de 10-12 repetições (cada perna).
>     *   **Dica FitNerd:** Dê um passo largo à frente, flexionando os joelhos até formar um ângulo de 90 graus. Mantenha o tronco reto e o abdômen contraído para equilibrar o corpo.
> 5.  **Levantamento Terra:** 3 séries de 5-8 repetições.
>     *   **Dica FitNerd:** Mantenha a coluna reta e use a força das pernas e do quadril para levantar a barra. Concentre-se em manter a postura correta para evitar lesões.
> 6.  **Extensora:** 3 séries de 10-12 repetições.
>     *   **Dica FitNerd:** Ajuste o equipamento para que seus joelhos fiquem alinhados com o eixo da máquina. Concentre-se em contrair o quadríceps durante a extensão da perna.
> 7.  **Flexora (deitado ou em pé):** 3 séries de 10-12 repetições.
>     *   **Dica FitNerd:** Ajuste o equipamento para que seus tornozelos fiquem posicionados sob o apoio. Concentre-se em flexionar a perna, sentindo a contração dos isquiotibiais.
> 8.  **Panturrilha em Pé (no Smith ou na máquina específica):** 4 séries de 15-20 repetições.
>     *   **Dica FitNerd:** Mantenha as pernas esticadas e eleve-se na ponta dos pés, sentindo a contração da panturrilha. Utilize uma amplitude completa do movimento.
> 
> **🔥 Técnicas Avançadas (1-2 exercícios por treino):**
> 
> *   **Drop-Set:** Ao final da última série, diminua a carga em 20-30% e faça o máximo de repetições que conseguir.
> *   **Isometria:** Mantenha a contração muscular no ponto mais difícil do exercício por 2-3 segundos.
> *   **Fase Excêntrica Controlada:** Diminua a velocidade na fase de descida do movimento (3-4 segundos).
> 
> **💧 Hidratação:** Beba água antes, durante e depois do treino.
> 
> **😴 Descanso:** Durma bem para que seus músculos se recuperem e cresçam.
> 
> **🍎 Alimentação:** Siga uma dieta equilibrada e rica em proteínas para otimizar os resultados.
> 
> **Lembrem-se:** A consistência é fundamental! Sigam o plano, ajustem as cargas e técnicas com a ajuda dos nossos profissionais e, em breve, vocês estarão ostentando pernas de aço! 💪🚀
> 
> **#FitNerd #TreinoDePernas #Hipertrofia #Força #VemMonstro**
> 


--------------------------------------------------------------

--- 📝 Resultado do Agente 4 (Revisor) ---



> O plano de treino está ótimo e pronto para ser gerado!


--------------------------------------------------------------
